# All NASDAQ ANALYSIS

In [ ]:
#libraries
import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import talib as ta
import seaborn as sns
from datetime import datetime
sns.set_theme()

pd.set_option('display.max_columns', None);pd.set_option('display.max_rows', None)

In [ ]:
def plot_with_mpl(root,tick,day=60,sty='classic',typ='candle'):
    "returns the candle stick of a stock tick for the given root file."

    df=pd.read_csv(root+'/'+tick+'.csv').tail(day)
    df['Date']=df['Date'].apply(lambda x:x.split()[0])

    df['rsi']=ta.RSI(df['Close']);df['sma-5']=ta.SMA(df['Close'],5);df['sma-10']=ta.SMA(df['Close'],10)
    #df.fillna(0,inplace=True)
    apds = [ 
         mpf.make_addplot((df['rsi']),panel=1,color='r',secondary_y=True,type='line',width=0.5),
         #mpf.make_addplot((df['sma-5']),color='g',type='line',width=0.2),
         #mpf.make_addplot((df['sma-10']),color='r',type='line',width=0.2)
       ]
    df.Volume=df.Volume/10**6              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,axes=mpf.plot(df,type=typ,mav=(5,21),volume=True,tight_layout=False,figratio=(90,36),figscale=1,
                    style=sty,returnfig=True,addplot=apds,scale_width_adjustment=dict(volume=0.5,candle=0.8,lines=0.4))
    axes[0].legend([" "," ","5-SMA","21-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    axes[0].set_title(tick)
    mpf.show()

def make_clickable(url, name):
    return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)

#df['link'] = df.apply(lambda x: make_clickable(x['url'], x['name']), axis=1)

In [ ]:


def compare_bars(temp_df,rows=30,title=""):
    choose=['TICKER','VOL','%CHG','%ATR','CP']
    plot_df=temp_df[choose].sort_values(by='VOL',ascending=False).reset_index(drop=True,).head(rows)
    columns=plot_df.columns.to_list()
    x=columns[0];y=columns[1:]
    today=datetime.now().strftime("%m/%d/%Y")
    f, axs = plt.subplots(2, 2, figsize=(16, 10), gridspec_kw=dict(width_ratios=[4, 4]))
    cols=axs.flat
    for i,j in enumerate(y):
        g=sns.barplot(data=plot_df,x=x,y=y[i],ax=cols[i])
        for item in g.get_xticklabels():
            item.set_rotation(90)
            item.set_fontsize(10)
            item.set_fontweight('bold')
        for i in g.containers:
            g.bar_label(i,rotation=90,fontsize=10,fontweight='bold',color='b',padding=5)
    f.suptitle(f"{title.upper()} {today}")   
    f.tight_layout()
    f.savefig(title+'.pdf')
    #f.show()

In [ ]:
#re-set index
def reset_index(df):
    df.index=np.arange(1,df.shape[0]+1)
    return df

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]


def highlight_min(s):
    '''
    highlight the minimum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['background-color: red' if v else '' for v in is_min]

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

In [ ]:
dff=pd.read_csv('/home/thakur/test/sector/XLF.csv')#,index_col='Date')
dff.tail()


In [ ]:
sectors=['Technology', 'Health Care', 'Consumer Discretionary', 'Finance',
'Energy', 'Consumer Staples', 'Industrials', 'Utilities',
       'Real Estate', 'Telecommunications', 'Miscellaneous',
       'Basic Materials']

In [ ]:
#all sectors
#df.style.highlight_max(color = 'lightgreen', axis = 0)

df_sector=pd.read_csv('/home/thakur/test/combined/all_sectors.csv')
st_df=pd.read_csv("/home/thakur/stock_information/sectors.csv")  #11 sectors
st_list=st_df["TICKER"].to_list()
df_sector['SECTOR']=st_df.SECTOR.to_list()
df_sector['SECTOR'] = df_sector['SECTOR'].apply(lambda x:x.replace(' ', '_'))
df_sector['%ATR']=round(100*df_sector['AVG10ATR']/df_sector['CP'],2)
df_sector=df_sector.sort_values(by='RSI',ascending=False)
df_sector=reset_index(df_sector)

#df_sector.style.apply(highlight_max)

# df_sector.style.\
#     applymap(color_negative_red).\
#     format("{:.2}")
df_sector

In [ ]:
compare_bars(df_sector,title='sector')

In [ ]:
# df_sector.style.\
#     applymap(color_negative_red).\
#     format("{:.2}")

df_sector.style.apply(highlight_max, subset=['RSI', 'CP', 'VOL','%CHG','%ATR'])\
.apply(highlight_min, subset=['RSI', 'CP', 'VOL','%CHG','%ATR'])\
#.format({'VOL':"{:.2f}",'CP':"{:.2f}",'RSI':"{:.2f}"})

#.style.format({'VOL':'{:.2f}'})
#df_sector.style.apply(highlight_min, subset=['RSI', 'CP', 'VOL','%CHG','%ATR'])
#df_sector.style.format({'VOL':"{:.2f}",'CP':"{:.2f}",'RSI':"{:.2f}"})
#cols=['CP']#,'RSI','SMA5','SMA21','SMA50','SMA50','SMA200','VOL','AVGVOL','%CHG','AVG10ATR','%ATR']
#df_sector[cols]=df_sector[cols].applymap('{0:.2f}'.format)
#df_sector
#df_sector['CP'].style.format('{:,.3f}')

In [ ]:
#plotting the sectors

tick=df_sector['TICKER'].to_list()
sec=df_sector['SECTOR'].to_list()
root='/home/thakur/test/sector'
for c,i in enumerate(tick):
    #file=root.format(cat[c],i)
    #print(f"file: {file}")
    print(f"\n\n{df_sector[df_sector['TICKER']==i].to_string()}")
    plot_with_mpl(root,i,sty='default')

In [ ]:
#dataframe which includes the information about the performance
df=pd.read_csv('/home/thakur/test/combined/all_nasdaq.csv')
df['IPO'].fillna(0,inplace=True);df['IPO'].astype(float)
df.fillna('N-A',inplace=True)
df=df[(df['%CHG']!=np.inf) & (df['%CHG']!='N/A') & (df['CP']>0)]
df['%ATR']=round(100*df['ATR']/df['CP'],2)
df['SECTOR'] = df['SECTOR'].apply(lambda x:x.replace(' ', '_'))

df.head()

# NASDAQ BEST

In [ ]:
# HH, HL, HC
temp_df=df.copy()\
.query('HH & HL & HC & HV & (CP>SMA50) & (CP>SMA200) & (`%CHG`>0) & VOL>0.5') \
.sort_values(by='CP',ascending=True)\
.replace(['micro','nano','mega','large'],np.nan,inplace=False)\
.pipe(lambda x:x.loc[(x.COUNTRY=='United States')])\
.dropna()\
.drop(['LL','LH','SMA21','HL','HH','HC','HV','COUNTRY'],axis=1)\
.reset_index(drop=True)\


temp_df

In [ ]:
compare_bars(temp_df,title='best_nasdaq')

In [ ]:
ticker='LXRX'
root=temp_df.loc[temp_df.TICKER==ticker,]['CATEGORY'].values[0]
root

In [ ]:
root='/home/thakur/test/{}'
plot=True

tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
if plot:
    for c,i in enumerate(tick):print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')

In [ ]:
df.SECTOR.unique()
# df_test=df.sort_values(by='%CHG')
# df_test

In [ ]:
df.SECTOR.value_counts()
# #column names
# df.dtypes

In [ ]:
countries=df['COUNTRY'].value_counts()
countries


In [ ]:
countries=list(df['COUNTRY'].unique())
countries

# DOW JONES & SNP-500


In [ ]:
dow=pd.read_csv('/home/thakur/stock_information/dow.csv')['Symbol'].to_list()
snp=pd.read_csv('/home/thakur/stock_information/snp500finalwvol.csv')['SYMBOL'].to_list()

In [ ]:
temp_df=df[df['TICKER'].isin(snp)].copy()
#temp_df


# DOW JONES

In [ ]:
#%%script false --no-raise-error
yahoo='https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'
fmt=33*'=='
fmt1=35*'**'
choose=['%CHG','VOL','%ATR','RSI']
how=False
plot=False #False
show_number=10

temp_df=df[df['TICKER'].isin(dow)].copy()

temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))
temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))

root='/home/thakur/test/{}'
columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'CATEGORY']
temp_df=temp_df[columns_selected]
for ch in choose:
    temp_df=temp_df.sort_values(by=ch,ascending=how).head(30)
    #temp_df['yahoo']=temp_df['TICKER'].apply(lambda x:yahoo.format(x,x))
    #temp_df['link'] = temp_df['TICKER'].apply(lambda x: make_clickable(yahoo.format(x,x),x))
    temp_df=reset_index(temp_df)

    print(f"\n\n{fmt} [ { ch } ] {fmt} \n\n {temp_df.to_string()}\n\n")#{fmt}{fmt}")
    
    tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
    if plot and ch==choose[-1]:
        for c,i in enumerate(tick):print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')

In [ ]:
compare_bars(temp_df,title='dow')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#from random import randint
#from mpl_toolkits.mplot3d import Axes3D

# Fixing random state for reproducibility
#np.random.seed(19680801)


fig = plt.figure(figsize=(25,15))
ax = fig.add_subplot(projection='3d')

n=30 #30 different colors

#colors_ = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))

# def get_cmap(n, name='hsv'):
#     '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
#     RGB color; the keyword argument name must be a standard mpl colormap name.'''
#     return plt.cm.get_cmap(name, n)

temp_df=temp_df.sort_values(by='VOL')

col = []
dx = .10 * np.ones(n)
dy = .10 * np.ones(n)
z=np.zeros(n)

# for i in range(n):
#    col.append('#%06X' % randint(0, 0xFFFFFF))

colors = ['r', 'g', 'b', 'y']
yticks = [0,2,1,3]
choose=['VOL','%CHG','%ATR','RSI']
for i, (c, k) in enumerate(zip(colors, yticks)):
    # Generate the random data for the y=k 'layer'.
    #xs = np.arange(20)
    xs=temp_df['TICKER'].to_list()
    #ys = np.random.rand(30)
    ys = temp_df[choose[i]].to_list()

    # You can provide either a single color or an array with the same length as
    # xs and ys. To demonstrate this, we color the first bar of each set cyan.
    cs = [c] * len(xs)
    #cs[0] = 'c'

    # Plot the bar graph given by xs and ys on the plane y=k with 80% opacity.
    ax.bar(xs, ys, zs=k, zdir='y', color=cs, alpha=0.8)
    #ax.set_xlabel('TICKERS',labelpad=0)
    #ax.bar3d(x=np.arange(30),y=k,z=z,dx=dx,dy=dy,dz=ys)

#ax.set_xlabel('TICKERS',labelpad=10)
ax.set_zlabel('VALUES')
ax.tick_params(axis='x', rotation=90,labelsize='small',pad=0)
ax.set_yticklabels(choose)
# ax.set_xticklabels(xs)
fig.align_xlabels()

# labelx = -0.5  # axes coords

# ax.xaxis.set_label_coords(labelx, 0.5)

# On the y axis let's only label the discrete values that we have data for.
ax.set_yticks(yticks)

plt.show()

# SNP-500

In [ ]:
#%%script false --no-raise-error
fmt=30*'=='
fmt1=35*'**'
choose=['%CHG','%CHG','VOL','RSI','RSI','%ATR','CP','CP']
how=False
plot=False
show_number=30

temp_df=df[df['TICKER'].isin(snp)].copy()

temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))
temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))

root='/home/thakur/test/{}'
columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR',  'IPO', 'COUNTRY','CATEGORY']#,'INDUSTRY',]
temp_df=temp_df[columns_selected]
ch_count=0
for c,ch in enumerate(choose,start=0):
    how=False
    if c in [1,4,7]:how=True
    temp_dff=temp_df.copy().sort_values(by=ch,ascending=how).head(show_number)
    #if ch=='%CHG':how=~how
    #if ch_count==1:how=True
    temp_dff=reset_index(temp_dff)

    print(f"\n\n{fmt} [ { ch } ] {fmt} \n\n {temp_dff.to_string()}\n\n")#{fmt}{fmt}")
    tick=temp_dff['TICKER'].to_list();cat=temp_dff['CATEGORY'].to_list()
    if plot and c==2:
        for c,i in enumerate(tick):print(f"\n\n{temp_dff[temp_dff['TICKER']==i].to_string()}");print(yahoo.format(i,i));plot_with_mpl(root.format(cat[c]),i,sty='default')

In [ ]:
snp_df=df[df['TICKER'].isin(snp)].copy()
snp_df.head()

# SNP500 Best

In [ ]:
# HH, HL, HC
temp_df=snp_df.copy()\
.query('HH & HL & (CP>SMA50) & (CP>SMA200) & (`%CHG`>0) & VOL>0.5') \
.sort_values(by='CP',ascending=True)\
.replace(['micro','nano','mega','large'],np.nan,inplace=False)\
.pipe(lambda x:x.loc[(x.COUNTRY=='United States')])\
.dropna()\
.drop(['LL','LH','SMA21','HL','HH','HC','HV','COUNTRY'],axis=1)\
.reset_index(drop=True)\


temp_df

In [ ]:
compare_bars(temp_df,title='snp-best')

In [ ]:
plot=True
tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
if plot:
    for c,i in enumerate(tick):print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')

# SNP-500

In [ ]:
#%%script false --no-raise-error
#ignore_category=['nano','micro']
fmt=25*'=='
#fmt1=35*'**'
choose='%CHG'
how=False
plot=True
show_number=50

for c,country in enumerate(countries):
    if c!=0:continue
    #df_usa=df[(df.COUNTRY==country )]
    df_usa=df[df['TICKER'].isin(snp)].copy()
    #df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    #df_usa=df_usa.query('HH & HL & HC')
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
    
    sectors=list(df.SECTOR.unique())
    #print(f"sectors: {sectors}")
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for s,sector in enumerate(sectors,start=1):
        #print(f"sector: {sector}");
        temp_df=df_usa[df_usa['SECTOR']==sector].copy();#print(f"\n\n{temp_df.head(10).to_string()}")
        if temp_df.empty:continue
        temp_df=reset_index(temp_df)[columns_selected]
        temp_df=temp_df.sort_values(by=choose,ascending=how).head(show_number)
        temp_df=reset_index(temp_df)
#         temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
#         temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

#         temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        print(f"\n\n{fmt} SECTOR {s}: {sector.upper()} [ { choose } ] {fmt} \n\n {temp_df.to_string()}")
        compare_bars(temp_df,title="snp-500-"+sector)

        if plot:
            tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
            for c,i in enumerate(tick):
                print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')

# IPO

In [ ]:
#%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=25*'=='
fmt1=25*'**'
choose='IPO'
how=False
plot=False
show_number=10

for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {str(country).upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for s,sector in enumerate(sectors,start=1):
        temp_df=df_usa[df_usa.SECTOR==sector]
        if temp_df.empty:continue
        temp_df=reset_index(temp_df)[columns_selected]
        temp_df=temp_df.sort_values(by=choose,ascending=how).head(show_number)
        temp_df=reset_index(temp_df)
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        print(f"\n\n{fmt} SECTOR {s}: {sector.upper()} ({ choose }){fmt} \n\n {temp_df.to_string()}")
        #compare_bars(temp_df,sector)
        compare_bars(temp_df,title="nasdaq-ipo-"+sector)
        tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
        if plot:
            for c,i in enumerate(tick):
            #file=root.format(cat[c],i)
            #print(f"file: {file}")
                print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
    # df_usa.shape
    # df_usa.head()
    # df_usa.columns

# CP

In [ ]:
#%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=25*'=='
fmt1=25*'**'
choose='CP'
how=True
plot=False
show_number=30

for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {country.upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for s,sector in enumerate(sectors,start=1):
        temp_df=df_usa[df_usa.SECTOR==sector]
        if temp_df.empty:continue
        temp_df=reset_index(temp_df)[columns_selected]
        temp_df=temp_df.sort_values(by=choose,ascending=how).head(show_number)
        temp_df=reset_index(temp_df)
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        print(f"\n\n{fmt} SECTOR:{s} {sector.upper()} ({ choose }){fmt} \n\n {temp_df.to_string()}")
        #compare_bars(temp_df,title="snp-500-"+sector)
        compare_bars(temp_df,title="nasdaq-cp-"+sector)
        tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
        for c,i in enumerate(tick):
            #file=root.format(cat[c],i)
            #print(f"file: {file}")
            
            if plot:print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
    # df_usa.shape
    # df_usa.head()
    # df_usa.columns

# ALL NASDAQ BEST

In [ ]:
%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=32*'=='
fmt1=35*'**'
choose=['VOL','%CHG','%ATR','IPO','CP']
how=False
show_number=2 #how many to show
plot=True
for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {country.upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    #df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for s,sector in enumerate(sectors):
        temp_df=df_usa[df_usa.SECTOR==sector].copy() #this line prints only limited output
        if temp_df.empty:continue
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        temp_df=reset_index(temp_df)[columns_selected]
        for c,cho in enumerate(choose,start=1):
            print(f"\n\n{c}: {sector.upper()} => {cho}\n")
            if cho=='CP':how=True
            temp_df=temp_df.sort_values(by=cho,ascending=how).head(show_number)
            how=False
            if cho=='%CHG':temp_df1
            temp_df=reset_index(temp_df)
            
            print(f"\n\n{fmt} SECTOR:{s+1} {sector.upper()} ({ cho }) {fmt} \n\n {temp_df.to_string()}")
            tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
            if (plot and (c==5)):
                for c,i in enumerate(tick):print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
 

In [ ]:
%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=32*'=='
fmt1=35*'**'
choose=['VOL','%CHG','%ATR','IPO','CP']
how=False
show_number=2 #how many to show
plot=True
for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {country.upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    #df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for s,sector in enumerate(sectors):
        temp_df=df_usa[df_usa.SECTOR==sector].copy() #this line prints only limited output
        if temp_df.empty:continue
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        temp_df=reset_index(temp_df)[columns_selected]
        for c,cho in enumerate(choose,start=1):
            print(f"\n\n{c}: {sector.upper()} => {cho}\n")
            if cho=='CP':how=True
            temp_df=temp_df.sort_values(by=cho,ascending=how).head(show_number)
            how=False
            if cho=='%CHG':temp_df1
            temp_df=reset_index(temp_df)
            
            print(f"\n\n{fmt} SECTOR:{s+1} {sector.upper()} ({ cho }) {fmt} \n\n {temp_df.to_string()}")
            tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
            if (plot and (c==5)):
                for c,i in enumerate(tick):print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
 

In [ ]:
#%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=35*'=='
fmt1=35*'**'
choose='VOL'
how=False
show_number=10 #how many to show
plot=False
for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {country.upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    #df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for s,sector in enumerate(sectors):
        temp_df=df_usa[df_usa.SECTOR==sector] #this line prints only limited output
        if temp_df.empty:continue
        temp_df=reset_index(temp_df)[columns_selected]
        temp_df=temp_df.sort_values(by=choose,ascending=how).head(show_number)
        temp_df=reset_index(temp_df)
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        print(f"\n\n{fmt} SECTOR:{s+1} {sector.upper()} ({ choose }) {fmt} \n\n {temp_df.to_string()}")
        tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
        if plot:
            for c,i in enumerate(tick):print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
 

# HIGHEST +   %CHANGE

In [ ]:
%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=35*'=='
fmt1=35*'**'
cor=10*'=='
choose='%CHG'
how=False
show_number=10 #how many to show
plot=False
for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {country.upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    #df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df_sector.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for sector in sectors:
        temp_df=df_usa[df_usa.SECTOR==sector] #this line prints only limited output
        if temp_df.empty:continue
        temp_df=reset_index(temp_df)[columns_selected]
        temp_df=temp_df.sort_values(by=choose,ascending=how).head(show_number)
        temp_df=reset_index(temp_df)
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        print(f"\n\n{fmt} SECTOR: {sector.upper()} ( {choose} ){fmt} \n\n {temp_df.to_string()}")
        print(f"\n{fmt}{fmt}{cor}")
        tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
        for c,i in enumerate(tick):
            #file=root.format(cat[c],i)
            #print(f"file: {file}")
            
            
            if plot:print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
    # df_usa.shape
    # df_usa.head()
    # df_usa.columns

# HIGHEST -VE %CHANGE

In [ ]:
%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=35*'=='
fmt1=35*'**'
choose='%CHG'
how=True
show_number=10 #how many to show
plot=False
for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {country.upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    #df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df_sector.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for sector in sectors:
        temp_df=df_usa[df_usa.SECTOR==sector] #this line prints only limited output
        if temp_df.empty:continue
        temp_df=reset_index(temp_df)[columns_selected]
        temp_df=temp_df.sort_values(by=choose,ascending=how).head(show_number)
        temp_df=reset_index(temp_df)
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        print(f"\n\n{fmt} SECTOR: {sector.upper()} (-{ choose }) {fmt} \n\n {temp_df.to_string()}");#print(f"\n{fmt}{fmt}")
        tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
        for c,i in enumerate(tick):
            #file=root.format(cat[c],i)
            #print(f"file: {file}")
            
            if plot:print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
    # df_usa.shape
    # df_usa.head()
    # df_usa.columns

In [ ]:
%%script false --no-raise-error
ignore_category=['nano','micro']
fmt=35*'=='
fmt1=35*'**'
choose='%ATR'
how=False
show_number=10 #how many to show
plot=False
for c,country in enumerate(countries):
    if c!=0:continue
    print(f"\n\n{fmt1} COUNTRY: {country.upper()} {fmt1} \n\n")
    df_usa=df[(df.COUNTRY==country )]
    df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
    #df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
    df_usa=reset_index(df_usa)
    root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
    sectors=list(df_sector.SECTOR.unique())
    columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
    
    for sector in sectors:
        temp_df=df_usa[df_usa.SECTOR==sector] #this line prints only limited output
        if temp_df.empty:continue
        temp_df=reset_index(temp_df)[columns_selected]
        temp_df=temp_df.sort_values(by=choose,ascending=how).head(show_number)
        temp_df=reset_index(temp_df)
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))

        temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
        print(f"\n\n{fmt} SECTOR: {sector.upper()} ({ choose }) {fmt} \n\n {temp_df.to_string()}")
        tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
        for c,i in enumerate(tick):
            #file=root.format(cat[c],i)
            #print(f"file: {file}")
            
            if plot:print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}");plot_with_mpl(root.format(cat[c]),i,sty='default')
    # df_usa.shape
    # df_usa.head()
    # df_usa.columns

In [ ]:
%%script false --no-raise-error
country='United States'
ignore_category=['nano']

df_usa=df[(df.COUNTRY==country )]
df_usa=df_usa[~df.CATEGORY.isin(ignore_category)]
df_usa=df_usa.query('HH & HL & HC')#.sort_values(by=choose,ascending=False)
df_usa=reset_index(df_usa)
df_usa.shape
df_usa.head()
df_usa.columns

In [ ]:
#print by sector
%%script false --no-raise-error
%matplotlib inline
root='/home/thakur/test/{}'
#sectors=list(df.SECTOR.unique())
sectors=list(df_sector.SECTOR.unique())
columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR','%ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
fmt=35*'=='
choose='CP'
how=True
for sector in sectors:
    temp_df=df_usa[df_usa.SECTOR==sector]
    if temp_df.empty:continue
    temp_df=reset_index(temp_df)[columns_selected]
   
    temp_df=temp_df.sort_values(by=choose,ascending=how)
    temp_df=reset_index(temp_df)
    temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
    temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))
    
    temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
    print(f"\n\n{fmt} SECTOR: {sector.upper()} {fmt} \n\n {temp_df.to_string()}")
    tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
    for c,i in enumerate(tick):
        #file=root.format(cat[c],i)
        #print(f"file: {file}")
        print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}")
        plot_with_mpl(root.format(cat[c]),i,sty='default')
        
        #print(f"tick: {i} {cat[c]}\n")

In [ ]:
#print by sector
%%script false --no-raise-error
%matplotlib inline
root='/home/thakur/test/{}'
sectors=list(df.SECTOR.unique())
columns_selected=['TICKER', 'CP', 'RSI',
       'VOL', 'AVGVOL', '%CHG', 'ATR',
       'SECTOR', 'INDUSTRY', 'IPO', 'COUNTRY', 'CATEGORY']
fmt=35*'=='
choose='CP'
how=True
for sector in sectors:
    temp_df=df_usa[df_usa.SECTOR==sector]
    if temp_df.empty:continue
    temp_df=reset_index(temp_df)[columns_selected]
   
    temp_df=temp_df.sort_values(by=choose,ascending=how)
    temp_df=reset_index(temp_df)
    temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Computer Software: Prepackaged Software','Computer Software'))
    temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.replace('Radio And Television Broadcasting And Communications Equipment','Communications Equipment'))
    
    temp_df['INDUSTRY']=temp_df['INDUSTRY'].apply(lambda x:x.lstrip('Retail: '))
    print(f"\n\n{fmt} SECTOR: {sector.upper()} {fmt} \n\n {temp_df.to_string()}")
    tick=temp_df['TICKER'].to_list();cat=temp_df['CATEGORY'].to_list()
    for c,i in enumerate(tick):
        #file=root.format(cat[c],i)
        #print(f"file: {file}")
        print(f"\n\n{temp_df[temp_df['TICKER']==i].to_string()}")
        plot_with_mpl(root.format(cat[c]),i,sty='default')
        
        #print(f"tick: {i} {cat[c]}\n")

In [ ]:
#file location
file='/home/thakur/stock_information/nasdaq_all.csv'


In [ ]:
df.columns #name of columns

In [ ]:
%%script false --no-raise-error
df=pd.read_csv(file) #read the file in dataframe
df.fillna('N/A',inplace=True)
df['Market Cap']=df['Market Cap'].apply(lambda x:round(x*10**-6,2))
df['% Change']=df['% Change'].apply(lambda x:x.strip('%')).astype(float)
df['Last Sale']=df['Last Sale'].apply(lambda x:x.lstrip('$')).astype(float).round(2)
df['Name']=df['Name'].apply(lambda x: x.replace('Common Stock',''))
df['Name']=df['Name'].apply(lambda x: x.replace('Class A',''))

df.head()

In [ ]:
df.shape #shape of the dataframe

In [ ]:
#countries present
countries=list(df['Country'].unique())

In [ ]:

countries

In [ ]:
df['Country'].value_counts()

In [ ]:
#select a country 
country='India'
df_name='df_'+country
df_name=df[df.Country==country]

df_name=reset_index(df_name)
df_name

choose='% Change'
df_name=df_name.sort_values(by=choose,ascending=False)
df_name=reset_index(df_name)
df_name.dtypes

In [ ]:
#select a country 
%%script false --no-raise-error
choose='% Change'
for country in countries:
    #df_name='df_'+country
    df_name=df[df.Country==country]
    df_name=reset_index(df_name)
    #df_name
    df_name=df_name.sort_values(by=choose,ascending=False)
    df_name=reset_index(df_name)
    print(f"COUNTRY: {country}\n {df_name.head()}")
    #df_name.dtypes

In [ ]:
df_name.head(25)

In [ ]:
df_name.tail(25)[::-1]